# 1: Setup and Package Import

In [1]:
import openmc
# import openmc.plotter
import paramak
from cad_to_dagmc import CadToDagmc
from pathlib import Path
from IPython.display import Image, display # For displaying plots
import numpy as np
import matplotlib.pyplot as plt
import neutronics_material_maker as nmm

# --- Step 0: Setup ---
# Set path for cross-sections.
openmc.config['cross_sections'] = Path.home() / 'nuclear_data' / 'cross_sections.xml'
print(f"Set openmc.config['cross_sections'] to {openmc.config['cross_sections']}")

# --- NEW: Define and create the output directory ---
output_dir = Path.cwd().parent / "outputs"
print(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)
print(f"All outputs will be saved to: {output_dir}")

Set openmc.config['cross_sections'] to /home/d63657rp/nuclear_data/cross_sections.xml
/home/d63657rp/fusion-neutronics-modelling/outputs
All outputs will be saved to: /home/d63657rp/fusion-neutronics-modelling/outputs


# 2: Paramak Model

In [2]:
# --- 1. Global Component Dictionary ---
# Maps layer ID to ((R,G,B,A), "Nicer Name")
# This is now the single source of truth for colors and names.

COMPONENT_MAP = {
    'layer_1': ((0.4, 0.9, 0.4), "centre_column"),
    'layer_2': ((0.6, 0.8, 0.6), "magnet_shield"),
    'layer_3': ((0.1, 0.8, 0.6), "first_wall"),
    'layer_4': ((0.1, 0.1, 0.9), "breeder"),
    'layer_5': ((0.4, 0.4, 0.8), "rear_wall"),
    'plasma':  ((1.0, 0.7, 0.8, 0.6), "plasma"),
}

In [3]:
def basic_tokamak(paramak_colors, a = (100), t = 0.55):

    radial_build_list = [
        (paramak.LayerType.GAP, 10),
        (paramak.LayerType.SOLID, 30),
        (paramak.LayerType.SOLID, 50),
        (paramak.LayerType.SOLID, 10),
        (paramak.LayerType.SOLID, (a / 100) * (2/3) * 70),  # breeder
        (paramak.LayerType.SOLID, 20),
        (paramak.LayerType.GAP, 60),
        (paramak.LayerType.PLASMA, 300),
        (paramak.LayerType.GAP, 60),
        (paramak.LayerType.SOLID, 20),
        (paramak.LayerType.SOLID, (a / 100) * (2/3) * 110),  # breeder
        (paramak.LayerType.SOLID, 10),
    ]

    vertical_build_list = [
        (paramak.LayerType.SOLID, 15),
        (paramak.LayerType.SOLID, (a / 100) * (2/3) * 80),  # breeder
        (paramak.LayerType.SOLID, 10),
        (paramak.LayerType.GAP, 50),
        (paramak.LayerType.PLASMA, 700),
        (paramak.LayerType.GAP, 60),
        (paramak.LayerType.SOLID, 10),
        (paramak.LayerType.SOLID, (a / 100) * (2/3) * 40),  # breeder
        (paramak.LayerType.SOLID, 15),
    ]

    my_reactor = paramak.tokamak(
        radial_build=radial_build_list,
        vertical_build=vertical_build_list,
        triangularity=t,
        rotation_angle=180,
        colors=paramak_colors
    )

    reactor_material_tags = my_reactor.names()
    reactor_material_tags[2] = 'eurofer' # required to ensure that the EUROFER first wall is properly parsed
    

    
    

    return (my_reactor, reactor_material_tags, radial_build_list)


paramak_colors = {
    name: details[0] for name, details in COMPONENT_MAP.items()
}

my_reactor, reactor_material_tags, radial_build_list = basic_tokamak(paramak_colors)

my_reactor
print(reactor_material_tags)

['layer_1', 'layer_2', 'eurofer', 'layer_4', 'layer_5', 'plasma']


3: Conversion to DAGMC

In [ ]:
def converter_code(my_reactor, reactor_material_tags):
    print("Converting CAD to DAGMC...")
    converter = CadToDagmc()

    # Add the CadQuery assembly from the Paramak object
    converter.add_cadquery_object(
        my_reactor,
        material_tags=reactor_material_tags
    )

    h5m_filename = output_dir / "dagmc_reactor.h5m"

    # Export the DAGMC .h5m file
    converter.export_dagmc_h5m_file(
        filename=str(h5m_filename),
        max_mesh_size=20.0,
        min_mesh_size=2.0
    )
    print(f"Successfully created {h5m_filename}")

    return (h5m_filename)

h5m_filename = converter_code(my_reactor, reactor_material_tags)

Converting CAD to DAGMC...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 19 (Line)
Info    : [  0%] Meshing curve 9 (Line)
Info    : [  0%] Meshing curve 17 (Line)
Info    : [  0%] Meshing curve 25 (BSpline)
Info    : [  0%] Meshing curve 15 (Line)
Info    : [  0%] Meshing curve 23 (BSpline)
Info    : [  0%] Meshing curve 26 (BSpline)
Info    : [  0%] Meshing curve 22 (Circle)
Info    : [  0%] Meshing curve 34 (BSpline)
Info    : [  0%] Meshing curve 11 (Line)
Info    : [  0%] Meshing curve 37 (Circle)
Info    : [  0%] Meshing curve 3 (Line)
Info    : [  0%] Meshing curve 8 (Circle)
Info    : [  0%] Meshing curve 18 (Line)
Info    : [  0%] Meshing curve 41 (BSpline)
Info    : [  0%] Meshing curve 36 (BSpline)
Info    : [  0%] Meshing curve 40 (BSpline)
Info    : [  0%] Meshing curve 2 (Circle)
Info    : [  0%] Meshing curve 6 (Line)
Info    : [  0%] Meshing curve 24 (BSpline)
Info    : [  0%] Meshing curve 13 (Circle)
Info    : [  0%] Meshing curve 30 (BSpline)
Info    : [  0%

# 4: Define Materials

In [6]:
def materials_library(reactor_material_tags, e = 60, c = 0.3):
    # --- RESET AUTO IDs ---
    # This resets the internal counters for Material, Cell, and Surface IDs
    # to prevent them from increasing every time the cell is re-run.
    openmc.reset_auto_ids()
    # --------------------
    eurofer_mat = nmm.Material.from_library('eurofer')
    
    materials_list = []

    eurofer_mat = nmm.Material.from_library('eurofer')

    for tag in reactor_material_tags:
        # Create a new material with the *exact* name as the tag
        mat = openmc.Material(name=tag)

    # In a real model, you'd add specific elements based on the tag
    # The 'paramak.tokamak' builder tags the plasma as 'plasma'
    # and other layers as 'radial_layer_1', 'vertical_layer_1' etc.
        if tag == 'plasma':
            mat.add_nuclide('H3', 1.0, 'ao')
            mat.add_nuclide('H2', 1.0, 'ao')  # Dummy plasma
            mat.set_density('g/cm3', 1.0e-6)
        elif tag == 'layer_4':
            breeder = openmc.Material()
            breeder.add_element(
                'Li',
                0.17,
                percent_type='ao',
                enrichment=e,
                enrichment_target='Li6',
                enrichment_type='ao')
            breeder.add_element('Pb', 0.83, 'ao')
            breeder.set_density('g/cm3', 11)

            coolant_he = nmm.Material.from_library(
                'He',
                temperature=800,
                pressure=8000000
            ).openmc_material  

            coolant_h2o = nmm.Material.from_library(
                'H2O',
                temperature=600,
                pressure=15500000
            ).openmc_material

            coolant_n2 = nmm.Material.from_library(
                'nitrogen',
                temperature=600,
                pressure=1000000
            ).openmc_material

            enriched_water = openmc.Material()
            enriched_water.add_nuclide('H1', 2, 'ao')
            enriched_water.add_nuclide('O16', 1, 'ao')
            enriched_water.set_density('g/cm3', 0.6611374655967136)

            mat = openmc.Material.mix_materials(
                materials=[breeder,coolant_h2o], fracs=[1-c, c], percent_type='vo', name = 'layer_4'
            )

        elif tag == 'eurofer': #First Wall
            mat.add_element('Zr', 0.9809, 'ao') # Fusion Zirconium Allowy
            mat.add_element('Sn', 0.015, 'ao')
            mat.add_element('Fe', 0.002, 'ao')
            mat.add_element('Cr', 0.001, 'ao')
            mat.add_element('O', 0.0011, 'ao')
            mat.set_density('g/cm3', 6.56)

            #mat.add_element('V', 0.92, 'ao') # Fusion Vanadium Alloy
            #mat.add_element('Cr', 0.04, 'ao')
            #mat.add_element('Ti', 0.04, 'ao')
            #mat.set_density('g/cm3', 6.05)

            #mat.add_element('Si', 1, 'ao') # Fusion SiC material
            #mat.add_element('C', 1, 'ao')
            #mat.set_density('g/cm3', 2.7)

            #mat = eurofer_mat.openmc_material # For Eurofer

            

        else:
            # Eurofer 97, can't get library import to work
            mat.add_element('Fe', 1, 'ao')
            mat.set_density('g/cm3', 7.8)

        materials_list.append(mat)

    # Create the final Materials collection
    my_materials = openmc.Materials(materials_list)

    


    return (my_materials, materials_list)





my_materials, materials_list = materials_library(reactor_material_tags)



In [ ]:
print(materials_list)

# 5: Defining Geometry

In [7]:
max_boundary = 1200.0

def geometry_code(h5m_filename, max_boundary_input = 1200):
    

    # Create the DAGMC Universe
    dag_univ = openmc.DAGMCUniverse(
        filename=str(h5m_filename),
        auto_geom_ids=True)



    # Set reflective boundary on the Y=0 plane for the 180-degree sector
    min_x = -max_boundary_input
    max_x = max_boundary_input
    min_y = 0.0  # <--- This is the cut plane
    max_y = max_boundary_input
    min_z = -max_boundary_input
    max_z = max_boundary_input

    # Define the bounding surfaces
    x_min_surf = openmc.XPlane(min_x, boundary_type='vacuum')
    x_max_surf = openmc.XPlane(max_x, boundary_type='vacuum')
    y_min_surf = openmc.YPlane(min_y, boundary_type='reflective')  # <-- REFLECTIVE
    y_max_surf = openmc.YPlane(max_y, boundary_type='vacuum')
    z_min_surf = openmc.ZPlane(min_z, boundary_type='vacuum')
    z_max_surf = openmc.ZPlane(max_z, boundary_type='vacuum')

    # Define the bounding cell
    bounding_cell = openmc.Cell(
        name='bounding_cell',
        region=+x_min_surf & -x_max_surf & +y_min_surf & -y_max_surf & +z_min_surf & -z_max_surf,
        fill=dag_univ
    )

    my_geometry = openmc.Geometry([bounding_cell])
    
    return my_geometry

my_geometry = geometry_code(h5m_filename, max_boundary)

# 6: Settings and Source

In [8]:
def settings_code(radial_build_list):
    print("Defining OpenMC settings...")
    # Calculate the approx. major radius from the radial_build
    radial_build_values = [item[1] for item in radial_build_list]
    plasma_index = radial_build_list.index((paramak.LayerType.PLASMA, 300))
    plasma_inner_radius = sum(radial_build_values[:plasma_index])
    plasma_thickness = radial_build_values[plasma_index]
    approx_major_radius = plasma_inner_radius + (plasma_thickness / 2.0)

    print(f"Placing source at approximate major radius: {approx_major_radius} cm")

    # Nudge source slightly off the y=0.0 reflective boundary
    source_location = (approx_major_radius, 0.01, 0.0)

    my_source = openmc.IndependentSource()
    radius = openmc.stats.Discrete([approx_major_radius], [1])
    z_values = openmc.stats.Discrete([0], [1])
    angle = openmc.stats.Uniform(a=0., b=2* 3.14159265359)
    my_source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))
    my_source.angle = openmc.stats.Isotropic()
    my_source.energy = openmc.stats.muir(e0 = 14080000.0, m_rat = 5, kt = 20000.0)  # 14.1 MeV neutrons

    my_settings = openmc.Settings()
    my_settings.batches = 25
    my_settings.particles = 5000
    my_settings.run_mode = 'fixed source'
    my_settings.source = my_source


    tbr_cell_tally = openmc.Tally(name='tbr')
    tbr_cell_tally.scores = ['(n,Xt)']
    tbr_cell_tally.nuclides = ['Li6', 'Li7']

    mesh = openmc.RegularMesh()  # Need to check the qualities of this mesh
    mesh.dimension = [100, 100, 100]
    # x,y,z coordinates start at 0 as this is a sector model. May need to change.
    mesh.lower_left = [0, 0, -350]
    mesh.upper_right = [650, 650, 350]
    mesh_filter = openmc.MeshFilter(mesh)  # creating a mesh

    tbr_mesh_tally = openmc.Tally(name="tbr_on_mesh")
    tbr_mesh_tally.filters = [mesh_filter]
    tbr_mesh_tally.scores = ["(n,Xt)"]

    tallies = openmc.Tallies([tbr_cell_tally, tbr_mesh_tally])
    print("Settings defined.")
    return my_settings, tallies


my_settings, tallies = settings_code(radial_build_list)

Defining OpenMC settings...
Placing source at approximate major radius: 376.66666666666663 cm
Settings defined.


# 7: Run Simulation

In [ ]:
# Basic Simulation

!rm *.h5

print("Running OpenMC simulation...")
my_model = openmc.Model(
    geometry=my_geometry,
    materials=my_materials,
    settings=my_settings
)
model_path = my_model.run(cwd=str(output_dir))
print(f"Simulation run complete. Results in {model_path}")

# Single Simulation

In [9]:
tbr_values = []
tbr_errors = []


print(f"Cleaning output directory: {output_dir}")
extensions_to_remove = ['*.h5', '*.h5m', '*.xml', '*.out']
files_removed = 0
for ext in extensions_to_remove:
    for f in output_dir.glob(ext):
        try:
            f.unlink()
            files_removed += 1
        except OSError as e:
            print(f"Warning: Could not remove {f.name}. Error: {e}")
print(f"Removed {files_removed} old files.")

reactor, material_tags, radial_build_list = basic_tokamak(paramak_colors)
settings_i, tallies_i = settings_code(radial_build_list)
materials, materials_list = materials_library(material_tags)

my_model = openmc.Model(
    geometry=geometry_code(converter_code(reactor, material_tags)),
    materials=materials_list,
    settings=settings_i,
    tallies=tallies_i
)

print(f"Running OpenMC in: {output_dir}")
statepoint_file = my_model.run(cwd=output_dir)
print(f"Statepoint file created: {statepoint_file}")

    # --- 3. Load statepoint file using a 'with' statement ---
    #
    #    THIS IS THE CRITICAL FIX.
    #
    #    Using 'with' ensures the statepoint file is properly closed
    #    and the file lock is released *before* the loop continues
    #    to the next iteration.
print(f"Loading statepoint: {statepoint_file} to extract results...")
with openmc.StatePoint(statepoint_file) as sp:
    tbr_cell_tally = sp.get_tally(name='tbr')

    tbr_mean = tbr_cell_tally.mean.sum()
    tbr_std_dev = tbr_cell_tally.std_dev.sum()

    tbr_values.append(tbr_mean)
    tbr_errors.append(tbr_std_dev)

    # By this point, the statepoint.10.h5 file is closed.
print(f"TBR: {tbr_mean:.4f} +/- {tbr_std_dev:.4f}")
print("--- ITERATION COMPLETED ---\n\n")

df = tbr_cell_tally.get_pandas_dataframe()
df

Cleaning output directory: /home/d63657rp/fusion-neutronics-modelling/outputs
Removed 5 old files.
Defining OpenMC settings...
Placing source at approximate major radius: 376.66666666666663 cm
Settings defined.
Converting CAD to DAGMC...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 3 (Line)
Info    : [  0%] Meshing curve 6 (Line)
Info    : [  0%] Meshing curve 2 (Circle)
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [  0%] Meshing curve 13 (Circle)
Info    : [  0%] Meshing curve 4 (Line)
Info    : [  0%] Meshing curve 7 (Line)
Info    : [  0%] Meshing curve 11 (Line)
Info    : [  0%] Meshing curve 14 (Line)
Info    : [  0%] Meshing curve 8 (Circle)
Info    : [  0%] Meshing curve 37 (Circle)
Info    : [  0%] Meshing curve 9 (Line)
Info    : [  0%] Meshing curve 19 (Line)
Info    : [  0%] Meshing curve 32 (Circle)
Info    : [  0%] Meshing curve 23 (BSpline)
Info    : [  0%] Meshing curve 22 (Circle)
Info    : [  0%] Meshing curve 40 (BSpline)
Info    : [  0%] Meshing curv

Info    : Done meshing 2D (Wall 33.0098s, CPU 136.061s)
Info    : 43432 nodes 89321 elements
written DAGMC file /home/d63657rp/fusion-neutronics-modelling/outputs/dagmc_reactor.h5m
Successfully created /home/d63657rp/fusion-neutronics-modelling/outputs/dagmc_reactor.h5m
Running OpenMC in: /home/d63657rp/fusion-neutronics-modelling/outputs
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
               

,nuclide,score,mean,std. dev.
0,Li6,"(n,Xt)",1.428845,0.004758
1,Li7,"(n,Xt)",0.002255,0.000021


# Multiple Runs

In [ ]:
# Breeder Blanket Thickness Sweep
# (This script assumes 'output_dir' is a pathlib.Path object
#  defined in a previous cell)

print(f"--- Starting Breeder Blanket Thickness Sweep ---")
print(f"All simulation outputs will be written to: {output_dir}")

avg_thickenss = 75  # blanket thickness in centimeters, standard value of 75cm
triangularity = 0.55  # triangularity, standard value of 0.55

#enrichment_values = np.arange(5, 96, 5)
#thickness_values = thickness_multipliers * avg_thickenss/100
#triangularity_values = np.arange(0.05, 1.01, 0.05)
coolant_fraction = np.arange(0.00, 0.96, 0.05)
tbr_values = []
tbr_errors = []

for i in coolant_fraction:
    print(f"\n--- Running simulation for coolant fraction: {i} ---")

    # --- NEW: Aggressive cleanup of old files ---
    # This deletes files from the *previous* iteration to prevent conflicts,
    # as you requested.
    print(f"Cleaning output directory: {output_dir}")
    extensions_to_remove = ['*.h5', '*.h5m', '*.xml', '*.out']
    files_removed = 0
    for ext in extensions_to_remove:
        for f in output_dir.glob(ext):
            try:
                f.unlink()
                files_removed += 1
            except OSError as e:
                print(f"Warning: Could not remove {f.name}. Error: {e}")
    print(f"Removed {files_removed} old files.")
    # --- END OF CLEANUP ---

    # --- 1. Model Setup (Unchanged) ---
    reactor, material_tags, radial_build_list = basic_tokamak(paramak_colors)
    settings_i, tallies_i = settings_code(radial_build_list)
    materials, materials_list = materials_library(material_tags, c=i)

    my_model = openmc.Model(
        geometry=geometry_code(converter_code(reactor, material_tags)),
        materials=materials_list,
        settings=settings_i,
        tallies=tallies_i
    )

    # --- 2. Run simulation in the correct output directory (Unchanged) ---
    print(f"Running OpenMC in: {output_dir}")
    statepoint_file = my_model.run(cwd=output_dir)
    print(f"Statepoint file created: {statepoint_file}")

    # --- 3. Load statepoint file using a 'with' statement ---
    #
    #    THIS IS THE CRITICAL FIX.
    #
    #    Using 'with' ensures the statepoint file is properly closed
    #    and the file lock is released *before* the loop continues
    #    to the next iteration.
    print(f"Loading statepoint: {statepoint_file} to extract results...")
    with openmc.StatePoint(statepoint_file) as sp:
        tbr_cell_tally = sp.get_tally(name='tbr')

        tbr_mean = tbr_cell_tally.mean.sum()
        tbr_std_dev = tbr_cell_tally.std_dev.sum()

        tbr_values.append(tbr_mean)
        tbr_errors.append(tbr_std_dev)

    # By this point, the statepoint.10.h5 file is closed.
    print(f"TBR for triangularity {i}: {tbr_mean:.4f} +/- {tbr_std_dev:.4f}")
    print("--- ITERATION COMPLETED ---\n\n")

print("\n\n--- SWEEP COMPLETED ---")
print("coolant:", coolant_fraction)
print("TBR Values:", tbr_values)
print("TBR Errors:", tbr_errors)

In [ ]:
#thickness_values = thickness_multipliers * avg_thickenss/100


plt.plot(coolant_fraction, tbr_values, 'k.')
plt.errorbar(coolant_fraction, tbr_values, tbr_errors, fmt='k.')

plt.title('TBR with Varying Coolant Content')
plt.xlabel('Helium Content by Volume')
plt.ylabel('TBR')
plt.grid()

plt.show()